In [ ]:
# sentence similarity using tensorflow hub models

import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()
from collections import Counter
from scipy import spatial
import numpy as np
import os
import logging
import operator
import pickle as pk
import json
import tensorflow_hub as hub
logging.getLogger("tensorflow").setLevel(logging.WARNING)



class Unbox_sm(object):
    
    
    # loading universal sentence encoder 
    # to find similar sentences/paragraphs
    
    
    model_dict = {
                  "elmo": "https://tfhub.dev/google/elmo/" , 
                  "use_m": "https://tfhub.dev/google/universal-sentence-encoder/",
                  "use_l": "https://tfhub.dev/google/universal-sentence-encoder-large/"
                  }
    
    @staticmethod
    def load_model(model_type, version):
        
    #  model_dict[model_type] + str(version)
    # some tfhub models support '.load' and some '.Module' 

        if model_type == 'elmo':
            
            # elmo model 
            # version : 1, 2, 3
            module_url = "https://tfhub.dev/google/elmo/" + str(version)
            return hub.Module(module_url)
            
        elif model_type == 'use_m':
            sd_ver = ['1','2', 1, 2]
            if version in sd_ver:
                module_url = "https://tfhub.dev/google/universal-sentence-encoder/" + str(version)
                return hub.Module(module_url)
            else:

                module_url = "https://tfhub.dev/google/universal-sentence-encoder/" + str(version)
                return hub.load(module_url)
            
            # universal sentence encoder medium
            # version : 1 , 2 , 3 , 4
            
        elif model_type == 'use_l':
            
            sd_ver = ['1','2', '3', 1, 2,3 ]
            
            if version in sd_ver:
                module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/" + str(version)
                return hub.Module(module_url)
            else:

                module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/" + str(version)
                return hub.load(module_url)
            
            
            #universal sentence encoder large
            # version : 1 , 2 , 3 , 4, 5
    
    
    

    @staticmethod
    def get_embedding(model_type, version, dataset):
        
        model = Unbox_sm.load_model(model_type, version)
        
        similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
        similarity_message_encodings = model(similarity_input_placeholder)

        with tf.Session() as session:
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())

            message_embeddings = session.run(similarity_message_encodings,
                                            feed_dict={similarity_input_placeholder: dataset})
            
        
        return message_embeddings
    
    
    @staticmethod
    # cosine distance
    def similarity_score(vector_one, vector_2):
        result = 1 - spatial.distance.cosine(vector_one, vector_2)
        return result
    
    
    #preprocessing
    @staticmethod
    def preprocessing_stopwords(sentence):
        cleaned = [i.lower() for i in word_tokenize(sentence) if i not in stop]
        return " ".join(cleaned)
    
    
    
    
    @staticmethod
    def unbox_sm(query, config_file = False ):
        
        default_config = {
                          'model_type'     : 'use_l', 
                          'version'        : '5',
                          'preprocessing'  :  False
                         }
        
        
        if config_file:
            default_config.update(config_file)
            
        
        if default_config['preprocessing']:
            
            query_vector_a =  preprocessing_stopwords(query['query_a'].lower())
            query_vector_b =  preprocessing_stopwords(query_['query_b'].lower())
        else:
            query_vector_a =  query['query_a'].lower()
            query_vector_b =  query_['query_b'].lower()
            
        
        # get the embeddings
        combine_query = [query_vector_a, query_vector_b ]
        emb_s = get_embedding(default_config['model_type'], default_config['version'], dataset)
        
        # calculate the distance
        sm_value = similarity_score(emb_s[0], emb_s[1])
        
        query['similarity_value'] = sm_value
        return query
    
    @staticmethod
    def get_vectors(query, config_file = False ):
        
        default_config = {
                          'model_type' : 'use_l', 
                          'version'    : '5' ,
                          'preprocessing'  :  False
                         }
        
        if config_file:
            default_config.update(config_file)
        
        if default_config['preprocessing']:
            query = [preprocessing_stopwords(sentence.lower()) for sentence in query]
            
        
        # get the embeddings
        emb_s = get_embedding(default_config['model_type'], default_config['version'], query)
        
        
        
        output = {
                  'embeddings'      : np.array(emb_s), 
                  'total_queries'   : len(query), 
                  'total_embeddings': len(emb_s), 
                  'shape'           : np.array(emb_s).shape
                 }
        
    
    # pandas dataframe as output
    @staticmethod
    def get_bulk_vectors(query, config_file = False ):
        
        default_config = {
                          'model_type' : 'use_l', 
                          'version'    : '5' ,
                          'preprocessing'  :  False
                         }
        
        if config_file:
            default_config.update(config_file)
        
        
        if default_config['preprocessing']:
            query = [preprocessing_stopwords(sentence.lower()) for sentence in query]
        
        # get the embeddings
        emb_s = get_embedding(default_config['model_type'], default_config['version'], query)
        
        output_ = {
                  'embeddings'      : emb_s, 
                  'total_queries'   : len(query), 
                  'total_embeddings': len(emb_s), 
                  'shape'           : np.array(emb_s).shape,
                  'queries'         : query
                 }
        
        return pd.DataFrame(output_)

In [ ]:
querys = {'query_a': 'Hello how are you', 'query_b': 'Hello I am fine' }

result = Unbox_sm.unbox_sm(querys)

In [ ]:
def load_model(model_type, version):
        
#  model_dict[model_type] + str(version)
# some tfhub models support '.load' and some '.Module' 

    if model_type == 'elmo':

        # elmo model 
        # version : 1, 2, 3
        module_url = "https://tfhub.dev/google/elmo/" + str(version)
        return hub.Module(module_url)

    elif model_type == 'use_m':
        sd_ver = ['1','2', 1, 2]
        if version in sd_ver:
            module_url = "https://tfhub.dev/google/universal-sentence-encoder/" + str(version)
            return hub.Module(module_url)
        else:

            module_url = "https://tfhub.dev/google/universal-sentence-encoder/" + str(version)
            return hub.load(module_url)

        # universal sentence encoder medium
        # version : 1 , 2 , 3 , 4

    elif model_type == 'use_l':

        sd_ver = ['1','2', '3', 1, 2,3 ]

        if version in sd_ver:
            module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/" + str(version)
            return hub.Module(module_url)
        else:

            module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/" + str(version)
            return hub.load(module_url)


        #universal sentence encoder large
        # version : 1 , 2 , 3 , 4, 5
def get_embedding(model_type, version, dataset):
        
    model = Unbox_sm.load_model(model_type, version)

    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_message_encodings = model(similarity_input_placeholder)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())

        message_embeddings = session.run(similarity_message_encodings,
                                        feed_dict={similarity_input_placeholder: dataset})


    return message_embeddings

def similarity_score(vector_one, vector_2):
    result = 1 - spatial.distance.cosine(vector_one, vector_2)
    return result
    
    

def preprocessing_stopwords(sentence):
    cleaned = [i.lower() for i in word_tokenize(sentence) if i not in stop]
    return " ".join(cleaned)




def unbox_sm(query, config_file = False ):

    default_config = {
                      'model_type'     : 'use_l', 
                      'version'        : '5',
                      'preprocessing'  :  False
                     }


    if config_file:
        default_config.update(config_file)


    if default_config['preprocessing']:

        query_vector_a =  preprocessing_stopwords(query['query_a'].lower())
        query_vector_b =  preprocessing_stopwords(query['query_b'].lower())
        
        print(query_vector_a,query_vector_b)
    else:
        query_vector_a =  query['query_a'].lower()
        query_vector_b =  query['query_b'].lower()

    print(query_vector_a,query_vector_b)
    # get the embeddings
    combine_query = [query_vector_a, query_vector_b ]
    emb_s = get_embedding(default_config['model_type'], default_config['version'], dataset)

    # calculate the distance
    sm_value = similarity_score(emb_s[0], emb_s[1])

    query['similarity_value'] = sm_value
    return query

def get_vectors(query, config_file = False ):

    default_config = {
                      'model_type' : 'use_l', 
                      'version'    : '5' ,
                      'preprocessing'  :  False
                     }

    if config_file:
        default_config.update(config_file)

    if default_config['preprocessing']:
        query = [preprocessing_stopwords(sentence.lower()) for sentence in query]
        



    # get the embeddings
    emb_s = get_embedding(default_config['model_type'], default_config['version'], query)



    output = {
              'embeddings'      : np.array(emb_s), 
              'total_queries'   : len(query), 
              'total_embeddings': len(emb_s), 
              'shape'           : np.array(emb_s).shape
             }
    
    return output


# pandas dataframe as output

def get_bulk_vectors(query, config_file = False ):

    default_config = {
                      'model_type' : 'use_l', 
                      'version'    : '5' ,
                      'preprocessing'  :  False
                     }

    if config_file:
        default_config.update(config_file)


    if default_config['preprocessing']:
        query = [preprocessing_stopwords(sentence.lower()) for sentence in query]

    # get the embeddings
    emb_s = get_embedding(default_config['model_type'], default_config['version'], query)

    output_ = {
              'embeddings'      : emb_s, 
              'total_queries'   : len(query), 
              'total_embeddings': len(emb_s), 
              'shape'           : np.array(emb_s).shape,
              'queries'         : query
             }

    return output_

In [ ]:
# working
load_f = load_model('use_l',5)

In [ ]:
# working
dataset = ['Hello how are you', 'Hello I am fine', 'Hello how are you', 'Hello I am fine']
res = get_embedding('use_l',5, dataset)

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [ ]:
query = {'query_a': 'Hello how are you', 'query_b': 'Hello I am fine' }

der = unbox_sm(query, config_file = {
                      'model_type'     : 'use_l', 
                      'version'        : '5',
                      'preprocessing'  :  False
                     } )

In [ ]:
der

In [ ]:
der

In [ ]:
# bulk checking

datas = get_vectors(dataset)

In [ ]:
import pandas as pd
result_p = get_bulk_vectors(dataset)

In [ ]:

query = {'query_a': 'Hello how are you', 'query_b': 'Hello I am fine' }
embed = transformer_model.unbox_transformer_sm(query, config_file = {'preprocessing': True})

In [ ]:
dataset = ['Hello how are you', 'Hello I am fine', 'Hello how are you', 'Hello I am fine']

bulk = transformer_model.get_transformer_vectors(query = dataset)

In [ ]:
bulk

In [ ]:
from semantic_similarity.unbox_transformers_sm import Unbox_transformers
transformer_model = Unbox_transformers()
embed = transformer_model.unbox_transformer_sm(query = {
                                                        'query_a': 'Hello how are you', 
                                                        'query_b': 'Hello I am fine' 
                                                        }, 
                                               config_file = {'preprocessing': True})

In [ ]:
transformer_model.similarity_score(bulk['embeddings'][1],bulk['embeddings'][2])

In [1]:
from semantic_similarity import Unbox_transformers
transformer_model = Unbox_transformers()

embed = transformer_model.unbox_transformer_sm(query = {
                                                        'query_a': 'Hello how are you', 
                                                        'query_b': 'Hello I am fine' 
                                                        }, 
                                               config_file = {'preprocessing': True})
print(embed)

/home/ANANT/apal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ANANT/apal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ANANT/apal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ANANT/apal/.local/lib/python3.6/site-packages/tensorflow/python/framework

it's going to take some time, depend on your internet speed ... try base model if taking long time
model loaded
{'query_a': 'Hello how are you', 'query_b': 'Hello I am fine', 'similarity_value': 0.7853398323059082}


In [ ]:
from semantic_similarity import Unbox_tensorflow_hub
transformer_model = Unbox_tensorflow_hub()

embed = transformer_model.unbox_sm(query = { 'query_a': 'Hello how are you', 
                                             'query_b': 'Hello I am fine' 
                                            }, 
                                  config_file = {'preprocessing': True})
print(embed)